In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import numpy as np


In [2]:
import sys
sys.path.append('../')
from networks import *
from images_compose import *

In [9]:
config = TransformConfig()
composed_train = config.composed_train
composed_test = config.composed_test

In [10]:
#transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
#dataset = datasets.CIFAR10(root='mnist_data', download=True, transform=transform)

In [10]:
train_dataset =  datasets.CIFAR10(root='./data', train=True, download=True, transform = composed_train)
validation_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform = composed_test)

# Create train and validation batch for training
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=100)
validation_loader = torch.utils.data.DataLoader(dataset=validation_dataset, batch_size=100)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.get_device_name(torch.cuda.current_device()))

NVIDIA GeForce RTX 4060 Ti


In [13]:
def train_model(model, train_loader, validation_loader, n_epochs = 100):
    
    # Global variable
    criterion = nn.CrossEntropyLoss()
    learning_rate = 0.1
    optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
    
    N_test = len(validation_dataset)
    accuracy_list = []
    train_loss_list = []
    model = model.to(device)
    train_cost_list = []
    val_cost_list = []
    
    for epoch in range(n_epochs):
        train_COST = 0
        for x,y in train_loader:
            x = x.to(device)
            y = y.to(device)
            model.train()
            optimizer.zero_grad()
            z = model(x)
            loss = criterion(z,y)
            loss.backward()
            optimizer.step()
            train_COST+=loss.item()
            
        train_COST = train_COST/len(train_loader)
        train_cost_list.append(train_COST)
        correct = 0
        
        # Perform the prediction on the validation data
        val_COST = 0
        for x_test, y_test in validation_loader:
            model.eval()
            x_test = x_test.to(device)
            y_test = y_test.to(device)
            z = model(x_test)
            val_loss = criterion(z, y_test)
            _, yhat = torch.max(z.data, 1)
            correct += (yhat==y_test).sum().item()
            val_COST+=val_loss.item()
        
        val_COST = val_COST/ len(validation_loader)
        val_cost_list.append(val_COST)
            
        accuracy = correct / N_test
        accuracy_list.append(accuracy)
        
        print("--> Epoch Number : {}".format(epoch + 1),
            " | Training Loss : {}".format(round(train_COST,4)),
            " | Validation Loss : {}".format(round(val_COST,4)),
              " | Validation Accuracy : {}%".format(round(accuracy * 100, 2)))
        
    return accuracy_list, train_cost_list, val_cost_list

In [14]:
model = NN02()
accuracy_list_normal, train_cost_list, val_cost_list = train_model(model, train_loader, validation_loader)

--> Epoch Number : 1  | Training Loss : 1.6933  | Validation Loss : 1.4779  | Validation Accuracy : 46.99%
--> Epoch Number : 2  | Training Loss : 1.4329  | Validation Loss : 1.3246  | Validation Accuracy : 52.75%
--> Epoch Number : 3  | Training Loss : 1.3402  | Validation Loss : 1.2481  | Validation Accuracy : 55.66%
--> Epoch Number : 4  | Training Loss : 1.2901  | Validation Loss : 1.197  | Validation Accuracy : 57.57%
--> Epoch Number : 5  | Training Loss : 1.2502  | Validation Loss : 1.1685  | Validation Accuracy : 58.29%
--> Epoch Number : 6  | Training Loss : 1.2154  | Validation Loss : 1.1756  | Validation Accuracy : 58.28%
--> Epoch Number : 7  | Training Loss : 1.1884  | Validation Loss : 1.1216  | Validation Accuracy : 60.67%
--> Epoch Number : 8  | Training Loss : 1.1635  | Validation Loss : 1.1074  | Validation Accuracy : 60.8%
--> Epoch Number : 9  | Training Loss : 1.1411  | Validation Loss : 1.0765  | Validation Accuracy : 62.24%
--> Epoch Number : 10  | Training Loss 

In [15]:
def KFold_Model(model, dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform = composed_train)):
  
  k = 10
  kf = KFold(n_splits=k, random_state=56, shuffle=True)
  accuracies = []
  torch.manual_seed(56)

  for train_idx, test_idx in kf.split(dataset):
      train_loader = DataLoader(Subset(dataset, train_idx), batch_size=100, shuffle=True)
      test_loader = DataLoader(Subset(dataset, test_idx), batch_size=100, shuffle=False)

      criterion = nn.CrossEntropyLoss()
      optimizer = optim.SGD(model.parameters(), lr=0.01)
      model = model.to(device)
      # Training
      for epoch in range(50):
          model.train()
          for images, labels in train_loader:
              images, labels = images.to(device), labels.to(device)
              optimizer.zero_grad()
              outputs = model(images)
              loss = criterion(outputs, labels)
              loss.backward()
              optimizer.step()

      # Evaluation
      model.eval()
      correct, total = 0, 0
      with torch.no_grad():
          for images, labels in test_loader:
              images, labels = images.to(device), labels.to(device)
              outputs = model(images)
              _, predicted = torch.max(outputs, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      accuracy = 100 * correct / total
      accuracies.append(accuracy)
      print(f'Fold Accuracy: {accuracy}%')

  print(f'Mean Accuracy: {np.mean(accuracies)}%')
  print(f'Standard Deviation: {np.std(accuracies)}%')

Files already downloaded and verified


In [16]:
model = NN02()
KFold_Model(model)

Fold Accuracy: 62.8%
Fold Accuracy: 64.52%
Fold Accuracy: 69.52%
Fold Accuracy: 70.74%
Fold Accuracy: 71.94%
Fold Accuracy: 73.46%
Fold Accuracy: 75.8%
Fold Accuracy: 77.06%
Fold Accuracy: 75.92%
Fold Accuracy: 77.76%
Mean Accuracy: 71.95199999999998%
Standard Deviation: 4.889050623587367%
